# WorkShop 2

Let's play with pandas and answer "stakeholders" _questions_.

Problems $2-13$ rely on your `Problem 1` answer.

## Problem  1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of __energy_df__.

Keep in mind that this is an _Excel file_, and not a _comma separated values file_. Also, make sure to _exclude_ the footer and header information from the datafile. The first _two columns_ are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

```python
['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
```

Convert `Energy Supply` to _gigajoules_ (there are $1000000$ _gigajoules_ in a _petajoule_**). For all countries which have missing data (_e.g._ data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (_for use in later questions_):

```bash
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

There are also several countries with parenthesis in their name. Be sure to _remove these_, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with _Numeric digits_ in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a _csv_ containing countries' GDP from $1960$ to $2015$ from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame __gdp_df__. 

Make sure to _skip the header_, and _rename_ the following list of countries:

```bash
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finally, load the [Scimago Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their _journal contributions_ in the aforementioned area. Call this DataFrame __scimago_df__.

Join the _thrgoee datasets_: _energy\_df_, _gdp\_df_ and _scimago\_df_, into a new dataset (using the _intersection_ of _country names_). Use only the _last_ $10$ years ($2006-2015$) of __GDP data__ and only the _top 15 countries_ by __Scimagojr 'Rank'__ (_Rank 1 through 15_). 

The _index_ of this DataFrame should be the __name__ of the country, and the _columns_ should be:
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']`.

_This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank"._

In [380]:
#!pip install xlrd
#!pip install openpyxl
import pandas as pd
import numpy as np

def generate_energy_df():
    energy_df = pd.read_excel('assets/Energy Indicators.xls', skiprows=16, skipfooter=38, engine='xlrd' )
    energy_df = energy_df.drop(columns=["Unnamed: 0", "Unnamed: 1"])
    energy_df = energy_df.drop(index=0)
    energy_df = energy_df.reset_index(drop=True)
    energy_df.rename(columns={"Unnamed: 2" : "Country"}, inplace=True)
    energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
    energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)
    energy_df['Energy Supply'] = energy_df['Energy Supply'] * 1e6
    energy_df['Country'] = energy_df['Country'].str.replace(r'\d+', '', regex=True)
    country_replacements = {
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"
    }
    energy_df['Country'] = energy_df['Country'].replace(country_replacements)
    energy_df['Country'] = energy_df['Country'].str.replace(r'\s*\(.*\)\s*', '', regex=True)
    return energy_df

def get_GDP_df():

    country_replacements = {
    "Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
    }
    
    gdp_df = pd.read_csv('assets/world_bank.csv', skiprows=4)
    gdp_df['Country Name'] = gdp_df['Country Name'].replace(country_replacements)
    gdp_df = gdp_df[['Country Name'] + [str(year) for year in range(2006, 2016)]]
    gdp_df.rename(columns={'Country Name': 'Country'}, inplace=True)

    return gdp_df

def get_scimago_df():
    scimago_df = pd.read_excel('assets/scimagojr-3.xlsx', nrows=16)
    return scimago_df

def answer_one():
    
    energy_df = generate_energy_df()
    gdp_df = get_GDP_df()
    ranking_scimago = get_scimago_df()

    combined_df = pd.merge(energy_df, ranking_scimago, on='Country', how='inner')
    combined_df = pd.merge(combined_df, gdp_df, on='Country', how='inner')
    combined_df = combined_df.set_index('Country')
    combined_df = combined_df.sort_values(by='Rank')

    return combined_df

In [381]:
# example of test cases
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecat

In [ ]:
# space for proffesor tests

## Problem 2

The _Problem 1_ joined three datasets then reduced this to just the top $15$ entries. When you joined the datasets, but before you reduced this to the top $15$ items, how many entries did you lose?

_This function should return a single number._

In [133]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [382]:
def answer_two():
    energy_df = generate_energy_df()
    gdp_df = get_GDP_df()
    scimago_df = get_scimago_df()

    # LOS CONVIERTO EN SET Y HAGO | (UNION) PARA CALCULAR ENTRIES
    initial_entries = len(set(energy_df['Country']) | set(gdp_df['Country']) | set(scimago_df['Country']))

    combined_df = pd.merge(energy_df, gdp_df, on='Country', how='inner')
    combined_df = pd.merge(combined_df, scimago_df[['Country', 'Rank']], on='Country', how='inner')

    final_entries = combined_df.shape[0]
    lost_entries = int(initial_entries - final_entries)
    
    return lost_entries

In [383]:
# example of test cases
assert type(answer_two()) == int, "Q2: You should return an int number!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)


In [ ]:
# space for proffesor tests

## Problem 3

What are the top $15$ countries for average GDP over the last $10$ years?

_This function should return a Series named `avg_gdp` with $15$ countries and their average GDP sorted in descending order._

In [384]:
def answer_three():
    gdp_df = answer_one()
        
    gdp_df['avg_gdp'] = gdp_df[[str(year) for year in range(2006, 2016)]].mean(axis=1)
    ranking_gdp_df = gdp_df[['avg_gdp']].sort_values(by='avg_gdp', ascending=False).head(15)
    ranking_gdp_series = ranking_gdp_df['avg_gdp']
    
    return ranking_gdp_series


In [385]:
# example of test cases
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)


In [ ]:
# space for proffesor tests

# Problem 4

By how much had the GDP changed over the $10$ year span for the country with the $6-th$ largest average GDP?

_This function should return a single number._

In [386]:
def answer_four():
    ranking_gdp_series = answer_three()
    sixth_highest_country = ranking_gdp_series.index[5]
    gdp_df = get_GDP_df()
    gdp_df.set_index('Country', inplace=True)
    gdp_change = gdp_df.loc[sixth_highest_country, '2015'] - gdp_df.loc[sixth_highest_country, '2006']
    
    return float(gdp_change)

In [387]:
# example of test cases
assert type(answer_four()) == float, "Q4: You should return a single number!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)


In [ ]:
# space for proffesor tests


## Problem 5

What is the _mean energy supply per capita_?

_This function should return a single number._

In [388]:
def answer_five():
    
    energy_df = answer_one()
    mean_energy_supply_per_capita = energy_df["Energy Supply per capita"].mean()
    return float(mean_energy_supply_per_capita)


In [389]:
# example of test cases
assert type(answer_five()) == float, "Q5: You should return a single number!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)


In [300]:
# space for proffesor tests

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)


,Energy Supply,Energy Supply per capita,Renewable Electricity Production,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1.271910e+11,93.0,19.75491,1,127050,126767,597237,411683,4.70,138,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,9.083800e+10,286.0,11.57098,2,96661,94747,792274,265436,8.20,230,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,1.898400e+10,149.0,10.23282,3,30504,30287,223024,61554,7.31,134,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,7.920000e+09,124.0,10.60047,4,20944,20357,206091,37874,9.84,139,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,3.070900e+10,214.0,17.28868,5,18534,18301,34266,12422,1.85,57,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12


## Problem 6

What __country__ has the _maximum % Renewable_ and what is the _percentage_?

_This function should return a tuple with the name of the country and the percentage._

In [390]:
def answer_six():
    energy_df = answer_one()
    return (energy_df['Renewable Electricity Production'].idxmax(), energy_df['Renewable Electricity Production'].max())

In [391]:
# example of test cases
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecat

In [322]:
# space for proffesor tests

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)


,Energy Supply,Energy Supply per capita,Renewable Electricity Production,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1.271910e+11,93.0,19.75491,1,127050,126767,597237,411683,4.70,138,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,9.083800e+10,286.0,11.57098,2,96661,94747,792274,265436,8.20,230,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,1.898400e+10,149.0,10.23282,3,30504,30287,223024,61554,7.31,134,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,7.920000e+09,124.0,10.60047,4,20944,20357,206091,37874,9.84,139,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,3.070900e+10,214.0,17.28868,5,18534,18301,34266,12422,1.85,57,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12


## Problem 7

Create a new column that is the _ratio of Self-Citations to Total Citations_. 
What is the _maximum_ value for this new column, and what _country_ has the _highest ratio_?

_This function should return a tuple with the name of the country and the ratio._

In [392]:
def answer_seven():
    merged_df = answer_one()
    merged_df['Citation Ratio'] = merged_df['Self-citations'] / merged_df['Citations']
    return (merged_df['Citation Ratio'].idxmax() , merged_df['Citation Ratio'].max())


In [393]:
# example of test cases
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecat

In [ ]:
# space for proffesor tests

## Problem 8

Create a column that estimates the _population using Energy Supply_ and _Energy Supply per capita_. 
What is the _third_ most populous country according to this estimate?

_This function should return the name of the country._

In [394]:
def answer_eight():
    merged_df = answer_one()
    merged_df['Population'] = merged_df['Energy Supply'] / merged_df['Energy Supply per capita']
    sorted_population = merged_df.sort_values(by='Population', ascending=False)
   
    return sorted_population.iloc[2].name


In [395]:
# example of test cases
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)


In [ ]:
# space for proffesor tests

## Problem 9

Create a column that estimates the _number of citable documents per person_. 

What is the correlation between the _number of citable documents per capita_ and the _energy supply per capita_? Use the `.corr()` method, (Pearson's correlation).

_This function should return a single number._

In [396]:
def answer_nine():
    merged_df = answer_one()
    merged_df['Population'] = merged_df['Energy Supply'] / merged_df['Energy Supply per capita']
    merged_df['Citable docs per capita'] = merged_df['Citable documents'] / merged_df['Population']
    return merged_df['Citable docs per capita'].corr(merged_df['Energy Supply per capita'])

In [397]:
# example of test cases
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecat

In [ ]:
# space for proffesor tests

## Problem 10

Create a new column with a $1$ if the _country's % Renewable_ value is at or above the median for all countries in the _top 15_, and a $0$ if the _country's % Renewable_ value is below the median.

_This function should return a series named `high_renew` whose index is the country name sorted in ascending order of rank._

In [398]:
def answer_ten():
    merged_df = answer_one()
    top_15_df = merged_df.nsmallest(15, 'Rank')
    renewable_median = top_15_df['Renewable Electricity Production'].median()
    top_15_df['HighRenew'] = (top_15_df['Renewable Electricity Production'] >= renewable_median).astype(int)   
    return top_15_df.sort_values(by='Rank')['HighRenew']

In [399]:
# example of test cases
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)


In [344]:
# space for proffesor tests

1
0
0
0
1
1
1
0
1
0
1
1
0
0
1


C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)


## Problem 11

Use the following dictionary to group the _Countries by Continent_, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the _sum_, _mean_, and _std deviation_ for the estimated population of each country.

```python
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}
```

_This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`_

In [400]:
def answer_eleven():
    ContinentDict  = {
        'China':'Asia', 
        'United States':'North America', 
        'Japan':'Asia', 
        'United Kingdom':'Europe', 
        'Russian Federation':'Europe', 
        'Canada':'North America', 
        'Germany':'Europe', 
        'India':'Asia',
        'France':'Europe', 
        'South Korea':'Asia', 
        'Italy':'Europe', 
        'Spain':'Europe', 
        'Iran':'Asia',
        'Australia':'Australia', 
        'Brazil':'South America'
    }
    
    merged_df = answer_one()
    merged_df['Population'] = merged_df['Energy Supply'] / merged_df['Energy Supply per capita']
    #REEMPLAZA CADA VALOR DEL INDICE Y LO REEMPLAZA POR EL CORRESPONDIENTE
    merged_df['Continent'] = merged_df.index.map(ContinentDict)
    return merged_df.groupby('Continent')['Population'].agg(['size', 'sum', 'mean', 'std'])

In [401]:
# example of test cases
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecat

In [ ]:
# space for proffesor tests

## Problem 12

Cut (check `.cut()` from Pandas) _% Renewable_ into $5$ bins, but in the same column. Group _Top15_ by the _Continent_, as well as these new _% Renewable_ bins. How many countries are in each of these groups? You could check `.size()` as aggregation in group by.

_This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable` (in string datatype). Do not include groups with no countries._

In [402]:
def answer_twelve():
    ContinentDict = {
        'China':'Asia', 
        'United States':'North America', 
        'Japan':'Asia', 
        'United Kingdom':'Europe', 
        'Russian Federation':'Europe', 
        'Canada':'North America', 
        'Germany':'Europe', 
        'India':'Asia',
        'France':'Europe', 
        'South Korea':'Asia', 
        'Italy':'Europe', 
        'Spain':'Europe', 
        'Iran':'Asia',
        'Australia':'Australia', 
        'Brazil':'South America'
    }

    merged_df = answer_one()
    merged_df['Continent'] = merged_df.index.map(ContinentDict)
    merged_df['% Renewable bins'] = pd.cut(merged_df['Renewable Electricity Production'], bins=5)
    result = merged_df.groupby(['Continent', '% Renewable bins']).size()
    result = result[result > 0]
    
    return result

In [403]:
# example of test cases
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\783500806.py:23: FutureWarning: The default of observed=False is deprecated a

In [ ]:
# space for proffesor tests

## Problem 13

Convert the _Population Estimate_ series to a string with thousands separator (using _dots_). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12.345.678,90

_This function should return a series `pop_est` whose index is the country name and whose values are the population estimate string_

In [407]:
def answer_thirteen():
    merged_df = answer_one()
    
    merged_df['Population'] = merged_df['Energy Supply'] / merged_df['Energy Supply per capita']
    pop_est = merged_df['Population'].apply(lambda x: "{:,.2f}".format(x).replace(",", "X").replace(".", ",").replace("X", "."))
    return pop_est


In [408]:
# example of test cases
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"

C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply'] = energy_df['Energy Supply'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy_df['Energy Supply per capita'] = energy_df['Energy Supply per capita'].replace('...', np.NaN)
C:\Users\Rocnar\AppData\Local\Temp\ipykernel_13720\673596095.py:12: FutureWarning: Downcasting behavior in `replace` is deprecat

In [ ]:
# space for proffesor tests